In [76]:
import requests
import pandas as pd
import numpy as np
import json

In [77]:
counter = 0

In [129]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,19):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2003","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2004"},"errors":[],"results":20,"paging":{"current":1,"total":18},"response":[{"player":{"id":17746,"name":"A. Taylor","firstname":"Andy","lastname":"Taylor","age":35,"birth":{"date":"1986-03-14","place":"Blackburn","country":"England"},"nationality":"England","height":"183 cm","weight":"76 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/17746.png"},"statistics":[{"team":{"id":67,"name":"Blackburn","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/67.png"},"league":{"id":null,"name":"Premier League","country":null,"logo":null,"flag":null,"season":"2004-2005"},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":0,"bench":1},"shots":{"total":null,"on":null},"goals":{"total":0,"conceded":null,"assists":null,"saves":null},"passes":{"total":null,"key":null,"accuracy":null},"tackles":{"to

In [38]:
counter

29

In [39]:
#complete year array
for i in range(30,33):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2004","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2009"},"errors":[],"results":20,"paging":{"current":1,"total":32},"response":[{"player":{"id":468,"name":"V. \\u0106orluka","firstname":"Vedran","lastname":"\\u0106orluka","age":35,"birth":{"date":"1986-02-05","place":"Derventa","country":"Bosnia and Herzegovina"},"nationality":"Croatia","height":"192 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/468.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":null,"name":"Premier League","country":null,"logo":null,"flag":null,"season":"2009-2010"},"games":{"appearences":29,"lineups":29,"minutes":2556,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":3,"bench":1},"shots":{"total":null,"on":null},"goals":{"total":1,"conceded":null,"assists":null,"saves":null},"passes":{"total":null,"key":null,

In [130]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

18

In [133]:
len(jsonData[17]['response'])

1

In [134]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [135]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
finalPageItems

1

In [136]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

5 5 lastname
5 5 age
5 5 nationality
5 5 height
5 5 weight
5 5 injured
5 6 id
5 6 firstname
5 6 lastname
5 6 age
5 6 nationality
5 6 height
5 6 weight
5 6 injured
5 7 id
5 7 firstname
5 7 lastname
5 7 age
5 7 nationality
5 7 height
5 7 weight
5 7 injured
5 8 id
5 8 firstname
5 8 lastname
5 8 age
5 8 nationality
5 8 height
5 8 weight
5 8 injured
5 9 id
5 9 firstname
5 9 lastname
5 9 age
5 9 nationality
5 9 height
5 9 weight
5 9 injured
5 10 id
5 10 firstname
5 10 lastname
5 10 age
5 10 nationality
5 10 height
5 10 weight
5 10 injured
5 11 id
5 11 firstname
5 11 lastname
5 11 age
5 11 nationality
5 11 height
5 11 weight
5 11 injured
5 12 id
5 12 firstname
5 12 lastname
5 12 age
5 12 nationality
5 12 height
5 12 weight
5 12 injured
5 13 id
5 13 firstname
5 13 lastname
5 13 age
5 13 nationality
5 13 height
5 13 weight
5 13 injured
5 14 id
5 14 firstname
5 14 lastname
5 14 age
5 14 nationality
5 14 height
5 14 weight
5 14 injured
5 15 id
5 15 firstname
5 15 lastname
5 15 age
5 15 nationalit

10 1 nationality
10 1 height
10 1 weight
10 1 injured
10 2 id
10 2 firstname
10 2 lastname
10 2 age
10 2 nationality
10 2 height
10 2 weight
10 2 injured
10 3 id
10 3 firstname
10 3 lastname
10 3 age
10 3 nationality
10 3 height
10 3 weight
10 3 injured
10 4 id
10 4 firstname
10 4 lastname
10 4 age
10 4 nationality
10 4 height
10 4 weight
10 4 injured
10 5 id
10 5 firstname
10 5 lastname
10 5 age
10 5 nationality
10 5 height
10 5 weight
10 5 injured
10 6 id
10 6 firstname
10 6 lastname
10 6 age
10 6 nationality
10 6 height
10 6 weight
10 6 injured
10 7 id
10 7 firstname
10 7 lastname
10 7 age
10 7 nationality
10 7 height
10 7 weight
10 7 injured
10 8 id
10 8 firstname
10 8 lastname
10 8 age
10 8 nationality
10 8 height
10 8 weight
10 8 injured
10 9 id
10 9 firstname
10 9 lastname
10 9 age
10 9 nationality
10 9 height
10 9 weight
10 9 injured
10 10 id
10 10 firstname
10 10 lastname
10 10 age
10 10 nationality
10 10 height
10 10 weight
10 10 injured
10 11 id
10 11 firstname
10 11 lastnam

15 2 age
15 2 nationality
15 2 height
15 2 weight
15 2 injured
15 3 id
15 3 firstname
15 3 lastname
15 3 age
15 3 nationality
15 3 height
15 3 weight
15 3 injured
15 4 id
15 4 firstname
15 4 lastname
15 4 age
15 4 nationality
15 4 height
15 4 weight
15 4 injured
15 5 id
15 5 firstname
15 5 lastname
15 5 age
15 5 nationality
15 5 height
15 5 weight
15 5 injured
15 6 id
15 6 firstname
15 6 lastname
15 6 age
15 6 nationality
15 6 height
15 6 weight
15 6 injured
15 7 id
15 7 firstname
15 7 lastname
15 7 age
15 7 nationality
15 7 height
15 7 weight
15 7 injured
15 8 id
15 8 firstname
15 8 lastname
15 8 age
15 8 nationality
15 8 height
15 8 weight
15 8 injured
15 9 id
15 9 firstname
15 9 lastname
15 9 age
15 9 nationality
15 9 height
15 9 weight
15 9 injured
15 10 id
15 10 firstname
15 10 lastname
15 10 age
15 10 nationality
15 10 height
15 10 weight
15 10 injured
15 11 id
15 11 firstname
15 11 lastname
15 11 age
15 11 nationality
15 11 height
15 11 weight
15 11 injured
15 12 id
15 12 firstn

In [137]:
len(finalArr)

341

In [138]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[17746, 'Andy', 'Taylor', ..., None, None, None],
       [18032, 'Jon', 'Stead', ..., None, None, None],
       [18099, 'Darren', 'Carter', ..., None, None, None],
       ...,
       [155103, 'Jon', 'Macken', ..., None, None, None],
       [280628, 'Jamie', 'McClen', ..., None, None, None],
       [44297, 'Bradley Scott', 'Jones', ..., None, None, None]],
      dtype=object)

In [139]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,17746,Andy,Taylor,35,England,183 cm,76 kg,False,67,Blackburn,...,None,None,0,0,0,None,None,None,None,None
1,18032,Jon,Stead,38,England,191 cm,76 kg,False,67,Blackburn,...,None,None,6,0,0,None,None,None,None,None
2,18099,Darren,Carter,38,England,188 cm,78 kg,False,54,Birmingham,...,None,None,1,0,0,None,None,None,None,None
3,18241,Mark,Yeates,36,Republic of Ireland,175 cm,66 kg,False,47,Tottenham,...,None,None,1,0,0,None,None,None,None,None
4,19203,Ashley,Cole,41,England,176 cm,66 kg,False,42,Arsenal,...,None,None,7,0,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,82029,John,Welsh,38,England,170 cm,73 kg,False,40,Liverpool,...,None,None,0,0,0,None,None,None,None,None
337,82591,David Haydn,Raven,37,England,183 cm,73 kg,False,40,Liverpool,...,None,None,0,0,0,None,None,None,None,None
338,155103,Jon,Macken,45,Republic of Ireland,178 cm,78 kg,False,50,Manchester City,...,None,None,0,0,0,None,None,None,None,None
339,280628,Jamie,McClen,43,England,170 cm,None,False,34,Newcastle,...,None,None,0,0,0,None,None,None,None,None


In [140]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['teamname']=='Tottenham']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
3,18241,Mark,Yeates,36,Republic of Ireland,175 cm,66 kg,False,47,Tottenham,...,None,None,1,0,0,None,None,None,None,None
5,19289,Michael,Dawson,38,England,189 cm,79 kg,False,47,Tottenham,...,None,None,2,0,0,None,None,None,None,None
11,44499,Ahmed Hossam Hussein,Abdelhamid,30,Egypt,188 cm,83 kg,False,47,Tottenham,...,None,None,1,0,0,None,None,None,None,None
17,155420,Nicky,Eyre,35,England,178 cm,None,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None
49,105424,Robbie,Keane,38,Republic of Ireland,175 cm,72 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None
84,83259,Stephen,Kelly,38,Republic of Ireland,183 cm,77 kg,False,47,Tottenham,...,None,None,1,0,0,None,None,None,None,None
85,90517,Paul,Robinson,42,England,193 cm,90 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None
110,115953,Andy,Reid,39,Republic of Ireland,170 cm,75 kg,False,47,Tottenham,...,None,None,2,0,0,None,None,None,None,None
117,231014,Sean,Davis,42,England,178 cm,76 kg,False,47,Tottenham,...,None,None,4,0,0,None,None,None,None,None
139,231007,Mauricio,Taricco,48,Argentina,175 cm,73 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None


In [141]:
df['season'] = '2004/2005'

In [142]:
#Export the df to a csv
df.to_csv('data2004&2005.csv')

In [143]:
counter

88